# R_DLGA algorithm _part1

## Overview

The performance of data-driven partial differential equations lacks stability when dealing with complex situations such as sparse data with high noise, so the robust deep learning Genetic algorithm (R-DLGA) is proposed.

## Technical path

The specific process of R-DLGA to solve this problem is as follows:

1. Run train.py to train the neural network, generate metadata and compute derivatives and obtain potential terms through genetic algorithms
2. Modify candidates in the train_pinn.py dict directory based on Step 1
3. Run train_pinn.py to get the final result

This part1 implements the functional code of the first step. After running the notebook, modify the term calculation under src/util.py and the dict in part2 according to the results of the genetic algorithm.

## Introduce code packages

In [ ]:
import os
import argparse
import random
import time

from mindspore import nn, context, ops, set_seed
from mindspore import value_and_grad, jit, data_sink, save_checkpoint
from mindspore.amp import DynamicLossScaler, auto_mixed_precision

from mindflow.utils import load_yaml_config
from mindflow.cell import MultiScaleFCSequential

The following `src` packages can be downloaded from [research/r_dlga/src] (https://gitee.com/mindspore/mindscience/tree/master/MindFlow/applications/research/r_dlga/src).

In [ ]:
from src import create_dataset, evaluate, produce_meta_data
from src import gene_algorithm

set_seed(0)
random.seed(0)

Parameters are configured, where --case has three choices, "burgers" means training on burgers equation, "cylinder_flow" means training on cylinder_flow dataset of navier_stokes2D equation, "periodic_hill" indicates training on a mountain flow dataset for the Reynolds mean Navier-Stokes equation.

In [ ]:
parser = argparse.ArgumentParser(description="train r_glda")
parser.add_argument("--case", type=str, default="burgers", choices=["burgers", "cylinder_flow", "periodic_hill"],
                    help="choose burgers, cylinder_flow or periodic_hill")
parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Running in GRAPH_MODE OR PYNATIVE_MODE")
parser.add_argument("--device_target", type=str, default="Ascend", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'Ascend', 'GPU'")
parser.add_argument("--device_id", type=int, default=0,
                    help="ID of the target device")
parser.add_argument("--config_file_path", type=str,
                    default="./configs/burgers.yaml")
input_args = parser.parse_args()

context.set_context(mode=context.GRAPH_MODE if input_args.mode.upper().startswith("GRAPH")
                    else context.PYNATIVE_MODE,
                    device_target=input_args.device_target,
                    device_id=input_args.device_id)
print(
    f"Running in {input_args.mode.upper()} mode, using device id: {input_args.device_id}.")
use_ascend = context.get_context(attr_key='device_target') == "Ascend"
print(use_ascend)
print("pid:", os.getpid())

Determine the equation to be trained and load the yaml file

In [ ]:
# get case name
case_name = input_args.case

# load configurations
config = load_yaml_config(input_args.config_file_path)

## Create a dataset

The following are three examples of datasets. During the course of using this code, you can download any dataset you want to train and place it in the src folder.

train_dataset is used for training, inputs and label are used for authentication.

### burgers dataset

The burgers dataset is randomly sampled according to the solution domain, initial conditions and boundary value conditions, and the training dataset and test dataset are generated.

Download： [physics_driven/burgers_pinns/dataset](https://download.mindspore.cn/mindscience/mindflow/dataset/applications/physics_driven/burgers_pinns/dataset/)

### Cylinder_flow dataset

cylinder_flow dataset is used to sample the initial conditions and boundary conditions data of a standard cylindrical flow with Reynolds number 100, respectively. For the training dataset, the problem domain and time dimension of the plane rectangle are constructed, and the known initial conditions and boundary conditions are sampled. The test set is constructed based on the points in the existing flow field.

Download： [physics_driven/flow_past_cylinder/dataset](https://download.mindspore.cn/mindscience/mindflow/dataset/applications/physics_driven/flow_past_cylinder/dataset/)。

### Periodic_hill Dataset

The data format is npy of numpy and the dimension is [300,700, 10]. The first two dimensions are the length and width of the flow field respectively, and the last dimension contains (x, y, u, v, p, uu, uv, vv, rho, nu) a total of 10 variables. Where, x, y, u, v, p are the x coordinate, y coordinate, x direction velocity, y direction velocity, and pressure of the flow field respectively. uu, uv, vv Reynolds average statistics; rho is the fluid density and nu is the kinetic viscosity coefficient.

Download： [dataset/periodic_hill_2d](https://download.mindspore.cn/mindscience/mindflow/dataset/periodic_hill_2d/)

In [ ]:
dataset_config = config["dataset"]
model_config = config["model"]
optimizer_config = config["optimizer"]
epochs = optimizer_config["epochs"]
summary_config = config["summary"]
# create dataset for training and validating
train_dataset, inputs, label = create_dataset(case_name, dataset_config)

## Build the model

The training part of the neural network in this example uses a simple fully connected network with a depth of 5 layers and an excitation function of `sin` function.

In [ ]:
model = MultiScaleFCSequential(in_channels=model_config["in_channels"],
                               out_channels=model_config["out_channels"],
                               layers=model_config["layers"],
                               neurons=model_config["neurons"],
                               residual=model_config["residual"],
                               act=model_config["activation"],
                               num_scales=1)

## Model training

Use **MindSpore >= 2.0.0** version, you can use functional programming paradigm to train neural networks.

Data fit is used here to build the proxy model from a small amount of training data.

In [ ]:
optimizer = nn.Adam(model.trainable_params(),
                    optimizer_config["initial_lr"])

# set ascend
if use_ascend:
    loss_scaler = DynamicLossScaler(1024, 2, 100)
    auto_mixed_precision(model, model_config["amp_level"])
else:
    loss_scaler = None

save_ckpt_path = summary_config["save_checkpoint_epochs"]

# create ckpt dir
if not os.path.exists(os.path.abspath(save_ckpt_path)):
    os.makedirs(os.path.abspath(save_ckpt_path))

# define forward function
def forward_fn(data, label):
    prediction = model(data)
    loss = nn.MSELoss()(prediction, label)
    if use_ascend:
        loss = loss_scaler.scale(loss)
    return loss

# define gradient function
grad_fn = value_and_grad(
    forward_fn, None, optimizer.parameters, has_aux=False)

# define train_step
@jit
def train_step(data, label):
    loss, grads = grad_fn(data, label)
    if use_ascend:
        loss = loss_scaler.unscale(loss)
    loss = ops.depend(loss, optimizer(grads))
    return loss

# data sink
sink_process = data_sink(train_step, train_dataset, sink_size=1)
steps_per_epochs = train_dataset.get_dataset_size()
print(steps_per_epochs)
print(train_dataset)

print("----start training----")
# train loop for nn
for epoch in range(1, epochs + 1):
    time_beg = time.time()
    model.set_train(True)
    for _ in range(steps_per_epochs):
        step_train_loss = sink_process()

    # set model to eval mode
    model.set_train(False)

    if epoch % summary_config["validate_interval_epochs"] == 0:
        # current epoch loss
        print(
            f"epoch: {epoch} train loss: {step_train_loss} epoch time: {(time.time() - time_beg) * 1000 :.3f}ms")
        evaluate(model, inputs, label, config)

    # save checkpoint
    if epoch % summary_config["save_checkpoint_epochs"] == 0:
        ckpt_name = f"{case_name}_nn-{epoch + 1}.ckpt"
        save_checkpoint(model, os.path.join(
            summary_config["save_checkpoint_epochs"], ckpt_name))

...
epoch: 100 train loss: 0.24175334 epoch time: 518.875ms
    predict total time: 195.06287574768066 ms
    l2_error:  0.813042652321916
...
epoch: 200 train loss: 0.05484033  epoch time: 288.337ms
    predict total time: 2.847433090209961 ms
    l2_error:  0.3844950973176981
...
epoch: 29800 train loss: 0.00031119288 epoch time: 283.176ms
    predict total time: 2.6092529296875 ms
    l2_error:  0.03155383103803965
...
epoch: 29900 train loss: 0.00034680235 epoch time: 246.574ms
    predict total time:  1.9993782043457031 ms
    l2_error:  0.031960893801602344
...
epoch: 30000 train loss: 0.00030471644 epoch time: 255.045ms
    predict total time: 1.7092227935791016 
    l2_error:  0.03110467273470587
End-to-End total time: 6085.048846006393 s


## Generate metadata

The following metadata is generated under the current network according to the automatic differentiation mechanism

In [ ]:
# produce meta data
produce_meta_data(case_name, config)

load successfully
Meta data produced successfully


## Genetic algorithm

The next iteration of the genetic algorithm based on the metadata will output the right term of the prediction equation

In [ ]:
gene_algorithm(case_name, config)

The best one:  [[2], [0, 0], [2], [1]]The best coef:  [[ 1.29570466e-03][-2.43374822e-01][-5.22589630e-12][-3.62047983e-09]]The best MSE 0.46980834457278564left is:     [[0, 0], [0, 1], [0, 3], [0, 3]][0.46980834457278564, 0.6880833403821989, 0.7075666639554047, 0.7578930294986997, 0.7578930294986997, 0.7616924071536476, 0.7616924071536476, 0.7616924071536476, 0.765759259310983, 0.772105611239328, 0.7773331999849253, 0.7777517230742845, 0.7778460331634339, 0.7778913996376994, 0.7973053049490681, 0.8044220450718776, 0.8044642001995869, 0.8045025579988737, 0.8144802626401659, 0.8178883580454595, 0.8266673719127372, 0.8334237654374858, 0.8342321627765292, 0.837488567287119, 0.8442066546480582, 0.844267867848322, 0.8444561950449638, 0.8490651610921612, 0.85701691824813, 0.85701691824813, 0.85701691824813, 0.85701691824813, 0.85701691824813, 0.85701691824813, 0.85701691824813, 0.85701691824813, 0.85701691824813, 0.85701691824813, 0.85701691824813, 0.8572462772894673, 0.8572462772894673, 0.8